#`mountUmount(`<font size="3px" color="#01c968">`Gdrive`</font>`)`



In [0]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# **Torrent Client**


## <img src='https://i.imgur.com/DOoxSuO.png' height="38" alt="Simple Torrent" />



In [0]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://i.imgur.com/DOoxSuO.png' height="50" alt="netdata"/></center>
#@markdown <center><h3>SimpleTorrent is a a self-hosted remote torrent client.</h3></center><br>
#@markdown <center><h5>Auto upload to gdrive after task completed. if gdrive is mounted and if not working tick install_old_version.</h5></center><br>

import os
import time
import pathlib 
import urllib.request
from IPython.display import clear_output

# script version
Version = '1.3.0'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
Install_old_version = False  # @param {type:"boolean"}
Auto_UP_Gdrive = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
AUTO_MOVE_PATH = "/content/drive/My Drive"  # @param {type:"string"}
force_change_version = "" # https://github.com/cikgusuraya/simple-torrent/releases
rclone_DestinationPath = "" # @param {type:"string"}
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/cikgusuraya/" \
                "ErinRamli/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    runSh,
    findProcess,
    loadingAn,
    updateCheck,
    ngrok
)
# making enviroment for simple-torrent
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('torrents').mkdir(mode=0o777, exist_ok=True)

def generateCmd(src, dst):
  FAST_LIST = True
  PATH_RClone_Config = "/usr/local/sessionSettings"
  cmd = f'rclone move "{src}" "{dst}" ' \
    f'--config {PATH_RClone_Config}/rclone.conf ' \
    f'{"--fast-list" if FAST_LIST else ""} --user-agent "Mozilla" ' \
    '--transfers 20 --checkers 20 --drive-server-side-across-configs ' \
    '-c --buffer-size 256M --drive-chunk-size 256M ' \
    '--drive-upload-cutoff 256M --drive-acknowledge-abuse ' \
    '--drive-keep-revision-forever --tpslimit 95 --tpslimit-burst 40 ' \
    '--stats-one-line --stats=5s -v'
  return cmd


if Auto_UP_Gdrive:
  # hCode1 = "https://raw.githubusercontent.com/cikgusuraya/" \
  #           "ErinRamli/master/res/scripts/doneCMD.sh"
  # urllib.request.urlretrieve(hCode1, f'{HOME}/doneCMD.sh')
  data = """#!/bin/bash

dir=${CLD_DIR}
path=${CLD_PATH}
abp="${dir}/${path}"
type=${CLD_TYPE}

if [[ ${type} == "torrent" ]]; then
"""
  nUpload = """    
    
    #Upload to Gdrive
    #mkdir -p "%s/$(dirname "${path}")"
    mv "${abp}" "%s/${path}"

    """ % (AUTO_MOVE_PATH, AUTO_MOVE_PATH)
  
  rcloneUpload = """
    #You can also use rcone move file to remote
    %s
    """ % generateCmd(r"${abp}", rclone_DestinationPath)
  
  end = """
fi
"""
  data = data + (rcloneUpload if rclone_DestinationPath else nUpload) + end
  with open(pathDoneCMD, 'w') as w:
    w.write(data)
  os.chmod(pathDoneCMD, 0o755)
else:
  try:
    os.unlink(pathDoneCMD)
  except FileNotFoundError:
    pass
  
configPath = pathlib.Path('cloud-torrent.json')
configsdata = r"""
{{
  "AutoStart": true,
  "EngineDebug": false,
  "MuteEngineLog": true,
  "ObfsPreferred": true,
  "ObfsRequirePreferred": false,
  "DisableTrackers": false,
  "DisableIPv6": false,
  "DownloadDirectory": "downloads/",
  "WatchDirectory": "torrents/",
  "EnableUpload": true,
  "EnableSeeding": false,
  "IncomingPort": 50007,
  "DoneCmd": "{}/doneCMD.sh",
  "SeedRatio": 1.5,
  "UploadRate": "High",
  "DownloadRate": "Unlimited",
  "TrackerListURL": "https://trackerslist.com/best.txt",
  "AlwaysAddTrackers": true,
  "ProxyURL": ""
}}
""".format(HOME)
with open(configPath, "w+") as configFile:
  configFile.write(configsdata)
#####################################

if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
    !kill -9 -1 &
clear_output()

# Simple Torrent installing ...
loadingAn()
if not os.path.isfile("/usr/local/bin/cloud-torrent"):
    dcmd = "wget -qq https://raw.githubusercontent.com/cikgusuraya/" \
            "ErinRamli/master/res/scripts/" \
            "simpleCloudInstaller.sh -O /root/simpleCloudInstaller.sh"
    runSh(dcmd)
    if Install_old_version:
        runSh("bash /root/simpleCloudInstaller.sh 1.2.3")
    else:
        runSh("bash /root/simpleCloudInstaller.sh")

if force_change_version:
  runSh("pkill cloud-torrent")
  runSh(f"bash /root/simpleCloudInstaller.sh {force_change_version}")


#Opening cloud-torrent in background
if not findProcess("cloud-torrent", "cloud-torrent"):
  PORT = 4444
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    cmdC = f'cloud-torrent --port {PORT} ' \
        '-t "SimpleTorrent" ' \
        '-c cloud-torrent.json ' \
        '--host 0.0.0.0 --disable-log-time ' \
        '&'
    for run in range(10):    
      runSh(cmdC, shell=True)
      time.sleep(3)
      try:
        urllib.request.urlopen(f"http://localhost:{PORT}")
        break
      except:
        print("Error: Simple-Torrent not starting. Retrying ...")

# START_SERVER
clear_output()
Server = ngrok(
    TOKEN, USE_FREE_TOKEN, [['simple-torrent', 4444, 'http'], 
    ['peerflix-server', 4445, 'http']], 'us', 
    [f"{HOME}/.ngrok2/sTpS.yml", 4040]
).start('simple-torrent')

## <img src='https://biplobsd.github.io/RLabClone/img/title_qbittorrent.png' height="45" alt="qBittorrent"/> <font size=1px>_Enhanced Edition_</font>

In [0]:
# ============================= FORM ============================= #
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}

# ================================================================ #

import os, psutil, time, urllib.request
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/cikgusuraya/" \
                "ErinRamli/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    checkAvailable,
    loadingAn,
    ngrok,
    runSh,
    displayUrl,
    findProcess
)

def addUtils():
    if not checkAvailable("/usr/local/sessionSettings"):
        runSh("mkdir -p -m 777 /usr/local/sessionSettings")
    if not checkAvailable("/content/upload.txt"):
        runSh("touch /content/upload.txt")
    if not checkAvailable("checkAptUpdate.txt", userPath=True):
        runSh("apt update -qq -y")
        runSh("apt-get install -y iputils-ping")

def configTimezone(auto=True):
    if checkAvailable("timezone.txt", userPath=True):
        return
    if not auto:
        runSh("sudo dpkg-reconfigure tzdata")
    else:
        runSh("sudo ln -fs /usr/share/zoneinfo/Asia/Ho_Chi_Minh /etc/localtime")
        runSh("sudo dpkg-reconfigure -f noninteractive tzdata")

def uploadQBittorrentConfig():
    if checkAvailable("updatedQBSettings.txt", userPath=True):
        return
    runSh(
        "mkdir -p -m 666 /content/qBittorrent /root/.qBittorrent_temp /root/.config/qBittorrent"
    )
    runSh(
        "wget -qq https://raw.githubusercontent.com/cikgusuraya/ErinRamli/master/res/qbittorrent/qBittorrent.conf \
            -O /root/.config/qBittorrent/qBittorrent.conf"
    )


def prepareSession():
    if checkAvailable("ready.txt", userPath=True):
        return
    else:
        addUtils()
        configTimezone()
        uploadQBittorrentConfig()

def installQBittorrent():
    if checkAvailable("/usr/bin/qbittorrent-nox"):
        return
    else:
        try:
            runSh("add-apt-repository -y ppa:poplite/qbittorrent-enhanced")
            runSh("apt-get install qbittorrent-enhanced-nox -qq -y")
        except:
            raise Exception('Error installing qbittorrent-enhanced-nox')

def startQBService():
    prepareSession()
    installQBittorrent()
    if not findProcess("qbittorrent-nox", "-d --webui-port"):
        runSh(f"qbittorrent-nox -d --webui-port={QB_Port}")
        time.sleep(1)

QB_Port = 10001
loadingAn()
startQBService()

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = ngrok(
    TOKEN, USE_FREE_TOKEN, [['qbittorrent', QB_Port, 'http']], 'sa', 
    [f"{HOME}/.ngrok2/qbittorrent.yml", 4088]
).start('qbittorrent', displayB=False)
displayUrl(server, pNamU='QBT_enhanced : ')


## <img src='https://github.com/ronggang/transmission-web-control/raw/master/src/tr-web-control/logo.png' height="55" alt="Transmission-Web-Control"/> 

In [0]:
# ============================= FORM ============================= #
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}

# ================================================================ #

import os, psutil, time, urllib.request, pathlib
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/cikgusuraya/" \
                "ErinRamli/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    checkAvailable,
    loadingAn,
    ngrok,
    runSh,
    displayUrl,
    findProcess
)


loadingAn()
if not os.path.exists('/usr/bin/transmission-daemon'):
  os.makedirs('downloads', exist_ok=True)
  os.makedirs('tools/transmission/', exist_ok=True)
  runSh('apt install transmission-daemon')
  nTWC = "https://raw.githubusercontent.com/ronggang/" \
    "transmission-web-control/master/release/install-tr-control.sh"
  urllib.request.urlretrieve(nTWC, 'tools/transmission/trInstall.sh')
  runSh('bash tools/transmission/trInstall.sh auto')
  try:
    pathlib.Path('tools/transmission/trInstall.sh').unlink()
  except FileNotFoundError:
    pass

if not findProcess('transmission-daemon', '--no-watch-dir'):
  !transmission-daemon --no-watch-dir --config-dir tools/transmission \
    --port 9091 --download-dir /content/downloads/ --dht --utp --no-portmap \
    --peerlimit-global 9999 --peerlimit-torrent 9999 --no-global-seedratio \
    -u admin -v admin --auth


# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = ngrok(
    TOKEN, USE_FREE_TOKEN, [['vuze', 9595, 'http'], 
                            ['transmission', 9091, 'http']], 'sa', 
    [f"{HOME}/.ngrok2/transmissionVuze.yml", 4058]
).start('transmission', displayB=False)
displayUrl(server, pNamU='Transmission : ', 
           ExUrl=fr"http://admin:admin@{server['url'][7:]}",
           btc='r')

# **Tools**


## <img src='http://pic2.orsoon.com/2017/0719/20170719091439708.png' height="60" alt="aria2" />___aria2___

In [0]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='http://pic2.orsoon.com/2017/0719/20170719091439708.png' height="200" alt="aria2"/></center>
#@markdown <center><h3>aria2<br />The next generation download utility.</h3></center><br>
import os, pathlib, zipfile, re
import urllib.request
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
# Aria2 Service
Aria2_rpc = True  # @param {type:"boolean"}
Ariang_WEBUI = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
# OUTPUT_DIR = ""  # @param {type:"string"}
PORT = 8221
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/cikgusuraya/" \
                "ErinRamli/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")


from ttmg import (
    updateCheck,
    runSh,
    loadingAn,
    ngrok,
    CWD,
    displayUrl,
    findProcess
)

if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
    !kill -9 -1 &
clear_output()
loadingAn()

# Ariang SETUP
runSh('sudo apt install -y aria2')
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)

# github latest releases tag define
def latestTag(link):
  import re
  from urllib.request import urlopen
  htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
  return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]

# Downloading latest version of ariang
if not os.path.exists("ariang/index.html"):
  BASE_URL = r"https://github.com/cikgusuraya/AriaNg"
  LATEST_TAG = latestTag(BASE_URL)
  urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
              f'AriaNg-{LATEST_TAG}-AllInOne.zip'
  urllib.request.urlretrieve(urlF, 'ariang/new.zip')
  with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref:
      zip_ref.extractall('ariang')
  try:
    pathlib.Path('ariang/new.zip').unlink()
  except FileNotFoundError:
    pass

# START_ARIANG_WEBUI_AND_ARIA2_RPC
try:
  if not OUTPUT_DIR:
    OUTPUT_DIR = f"downloads/"
  elif not os.path.exists(OUTPUT_DIR):
    clear_output()
    print("Error: Your set path not found! Create path!")
    exx()
except:
  OUTPUT_DIR = f"{CWD}/downloads/"
if Aria2_rpc:
  if not findProcess("aria2c", "--enable-rpc"):
    try:
      trackers = urllib.request.urlopen(
          "https://trackerslist.com/best_aria2.txt"
      ).read().decode()
      cmdC = r"aria2c --enable-rpc --rpc-listen-port=6800 -D " \
            fr"-d {OUTPUT_DIR} " \
            r"-j 20 " \
            r"-c " \
            fr"--bt-tracker={trackers} " \
            r"--bt-request-peer-speed-limit=0 " \
            r"--bt-max-peers=0 " \
            r"--seed-ratio=0.0 " \
            r"--max-connection-per-server=10 " \
            r"--min-split-size=10M " \
            r"--follow-torrent=mem " \
            r" &"
      runSh(
          cmdC,
          shell=True
      )
    except:
      print("aria2 rpc not enabling. Try again later!")

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
Server = ngrok(
    TOKEN, USE_FREE_TOKEN, [['Aria2_rpc', 6800, 'http'], 
    ['Ariang', 8221, 'http']], 'jp', 
    [f"{HOME}/.ngrok2/aria2.yml", 5042]
)
if Aria2_rpc:
  data = Server.start('Aria2_rpc', displayB=False)
  Host = data['url'][7:] 
  port = "80"
  clear_output()
if Ariang_WEBUI:
  if Aria2_rpc:
    filePath = 'ariang/index.html'
    with open(filePath, 'r+') as f:
      read_data = f.read()
      f.seek(0)
      f.truncate(0)
      read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', 
                        f'rpcHost:"{Host}"', read_data)
      read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
      read_data = re.sub(r'rpcPort:"\d+."', f'rpcPort:"{port}"', read_data)
      f.write(read_data)
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    runSh(f"python3 -m http.server {PORT} &", shell=True, cd="ariang/")
 
  data2 = Server.start('Ariang', displayB=False)
  displayUrl(data2, pNamU='Ariang : ')

if Aria2_rpc:
  display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 20px;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>Aria2 rpc config<p><thead>        <tr>            <th>Protocol</th>            <th>Host</th>            <th>Port</th>        </tr>    </thead>    <tbody>        <tr>            <td>WebSocket</td><td>"""+Host+"""</td><td>"""+port+"""</td></tr></tbody></table><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="http://bit.ly/2qoOw0a">NB. How to setup this's config. [Click ME]</a></button><center>"""))


Protocol,Host,Port
WebSocket,bafa04b2.jp.ngrok.io,80


##<img src='https://raw.githubusercontent.com/biplobsd/Google-Colab-CloudTorrent/master/src/zip.png' height="30" alt="netdata"> __Extracte/Archive Zip/Tar/Rar__

In [0]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Extract Files
MODE = "UNZIP"  # @param ["UNZIP", "UNTAR", "UNRAR", "7Z"]
PATH_TO_FILE = ""  # @param {type:"string"}
extractPath = ""  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# ================================================================ #
import os, urllib.request
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/cikgusuraya/" \
                "ErinRamli/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    runSh,
    checkAvailable,
)

def extractFiles():
    global extractPath
    if ARCHIVE_PASSWORD:
      passADD = f'-P {ARCHIVE_PASSWORD}'
    else:
      passADD = ''
    if not extractPath:
      extractPath = "/content/extract"
    os.makedirs(extractPath, exist_ok=True)
    if MODE == "UNZIP":
        runSh('unzip '+passADD+f' "{PATH_TO_FILE}" -d "{extractPath}"', output=True)
    elif MODE == "UNRAR":
        runSh(f'unrar x "{PATH_TO_FILE}" "{extractPath}" '+passADD+' -o+', output=True)
    elif MODE == "UNTAR":
        runSh(f'tar -C "{extractPath}" -xvf "{PATH_TO_FILE}"', output=True)
    else:
        runSh(f'7z x "{PATH_TO_FILE}" -o{extractPath} '+passADD, output=True)


extractFiles()

In [0]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Archive Files
MODE = "ZIP" #@param ["ZIP", "TAR", "7Z"]
FILENAME = ""  # @param {type:"string"}
PATH_TO_FILE = ""  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# ================================================================ #
import os, urllib.request
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/cikgusuraya/" \
                "ErinRamli/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    runSh,
    checkAvailable,
)

def extractFiles():
    global FILENAME
    if MODE == "ZIP":
        if not FILENAME:
          FILENAME = "/content/NEW_FILE.ZIP"
        if ARCHIVE_PASSWORD:
          passADD = f'--password "{ARCHIVE_PASSWORD}"'
        else:
          passADD = ''
        runSh(f'zip {passADD} -r "{FILENAME}" "{PATH_TO_FILE}"', output=True)
    elif MODE == "TAR":
        if not FILENAME:
          FILENAME = "NEW_FILE"
        runSh(f'tar -zcvf "{FILENAME}.tar.gz" {PATH_TO_FILE}', output=True)
    else:
        if not FILENAME:
            FILENAME = "NEW_FILE"
        runSh(f'7z a "{FILENAME}.7z" {PATH_TO_FILE}', output=True)


extractFiles()

#`sysTools(`<font size="3px" color="#05ba02">`_['>ssh']`</font> , <font size="3px" color="#f8f800">`noVNC`</font> )

## <img src='http://icons.iconarchive.com/icons/blackvariant/button-ui-system-apps/1024/Terminal-icon.png' height="25" alt="ssh" />___SSH and noVNC___

In [0]:
#@markdown <h3>⬅️ Click Here to create ssh tunnel</h3>
#@markdown <br><center><img src='http://icons.iconarchive.com/icons/blackvariant/button-ui-system-apps/1024/Terminal-icon.png' height="200" alt="ssh"/></center>
#@markdown <center><h3>Forward ssh now</h3></center><br>
import os
import random
import string
import urllib.request
from IPython.display import HTML, clear_output
import time

#####################################
USE_FREE_TOKEN = False
CREATE_VNC = True # @param {type:"boolean"}
CREATE_SSH = True # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/cikgusuraya/" \
                "ErinRamli/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    runSh,
    loadingAn,
    ngrok,
    displayUrl,
    findProcess,
    CWD,
    textAn,
)

loadingAn()

# password ganarate
try:
  print(f"Found old password! : {password}")
except:
  password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

if CREATE_SSH:
  USE_FREE_TOKEN = False

#Set root password
if CREATE_SSH  and os.path.exists('/var/run/sshd') == False:
  #Setup sshd
  runSh('apt install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen')
  runW.system_raw("echo root:$password | chpasswd")
  os.makedirs("/var/run/sshd", exist_ok=True)
  runW.system_raw('echo "PermitRootLogin yes" >> /etc/ssh/sshd_config')
  runW.system_raw('echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config')
  runW.system_raw('echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc')
  runW.system_raw('echo "export LD_LIBRARY_PATH" >> /root/.bashrc')

  #Run sshd
  if not findProcess("/usr/sbin/sshd", command="-D"):
    runSh('/usr/sbin/sshd -D &', shell=True)

if CREATE_VNC:
  clear_output()
  #start = time.time()
  textAn("Wait for almost 2 minutes. It's doing for VNC ready...")
  os.makedirs(f'{HOME}/.vnc', exist_ok=True)
  runW.system_raw('add-apt-repository -y ppa:apt-fast/stable < /dev/null')
  runW.system_raw('echo debconf apt-fast/maxdownloads string 16 | debconf-set-selections')
  runW.system_raw('echo debconf apt-fast/dlflag boolean true | debconf-set-selections')
  runW.system_raw('echo debconf apt-fast/aptmanager string apt-get | debconf-set-selections')
  runW.system_raw('apt install -y apt-fast')
  runW.system_raw('apt-fast install -y xfce4 xfce4-goodies firefox tightvncserver')
  runW.system_raw(rf'echo "{password}" | vncpasswd -f > ~/.vnc/passwd')
  data = """
#!/bin/bash
xrdb $HOME/.Xresources
startxfce4 &
"""
  with open(f'{HOME}/.vnc/xstartup', 'w+') as wNow:
    wNow.write(data)
  os.chmod(f'{HOME}/.vnc/xstartup', 0o755)
  os.chmod(f'{HOME}/.vnc/passwd', 0o400)
  runSh('sudo vncserver &', shell=True)
  runSh(f'git clone https://github.com/novnc/noVNC.git {CWD}/noVNC')
  runSh("bash noVNC/utils/launch.sh --listen 6080 --vnc localhost:5901 &",
        shell=True)
  #end = time.time()
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
Server = ngrok(TOKEN, USE_FREE_TOKEN, [['ssh', 22, 'tcp'], 
               ['vnc', 6080, 'http']], 'in', 
               [f"{HOME}/.ngrok2/sshVnc.yml", 4455])
data = Server.start('ssh', displayB=False)

# output
Host,port = data['url'][7:].split(':')
data2 = Server.start('vnc', displayB=False)
clear_output()
if CREATE_VNC:
  displayUrl(data2, pNamU='noVnc : ',
            EcUrl=f'/vnc.html?autoconnect=true&password={password}')
if CREATE_SSH:
  display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 20px;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>Ssh config<p><thead>        <tr>        <th>Host</th>            <th>Port</th>        <th>Password</th> </tr>    </thead>    <tbody>        <tr><td>"""+Host+"""</td><td>"""+port+"""</td><td>"""+password+"""</td></tr></tbody></table><center><br><table class="zui-table blueBG"><thead>        <tr>        <th>Simple ssh command</th></tr>    </thead>    <tbody>        <tr><td>ssh root@"""+Host+""" -p """+port+"""</td></tr></tbody></table></center><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="http://bit.ly/2RDhV0c">NB. How to setup this's config. [Click ME]</a></button><center>"""))
